In [136]:
import os
from mcp.server.fastmcp import FastMCP
from mcp.server import Server
import json
import logging
from typing import Optional, Dict, Any
from datetime import datetime, timezone, timedelta
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from mcp.server.sse import SseServerTransport
import argparse
from starlette.routing import Mount, Route
from pydantic import BaseModel
from starlette.applications import Starlette
from starlette.requests import Request
import uvicorn
import os
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
from pydantic_ai import Agent
from pydantic_ai.mcp import MCPServerSSE


BASE_URL = f"http://localhost:8000/v1"

os.environ["BASE_URL"]    = BASE_URL
os.environ["OPENAI_API_KEY"] = "abc-123"   

print("Config set:", BASE_URL)

Config set: http://localhost:8000/v1


In [137]:
!curl http://localhost:8000/v1/models -H "Authorization: Bearer $OPENAI_API_KEY"

{"object":"list","data":[{"id":"Qwen3-30B-A3B","object":"model","created":1752987375,"owned_by":"vllm","root":"Qwen/Qwen3-30B-A3B","parent":null,"max_model_len":40960,"permission":[{"id":"modelperm-ec24940cc38f4b2a925131a24ab63cfb","object":"model_permission","created":1752987375,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

In [138]:
from pydantic_ai import Agent
from pydantic_ai.mcp import MCPServerSSE

server = server = MCPServerSSE('http://localhost:8090/sse') 

In [139]:
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider

provider = OpenAIProvider(
    base_url=os.environ["BASE_URL"],
    api_key=os.environ["OPENAI_API_KEY"],
)

agent_model = model = OpenAIModel("Qwen3-30B-A3B", provider=provider)

In [140]:
from datetime import datetime
from zoneinfo import ZoneInfo  # Python 3.9+
dt = datetime.now(ZoneInfo("Asia/Kolkata"))

In [141]:
# # Define response model for meeting extraction
# from pydantic import BaseModel
# class MeetingTimeframe(BaseModel):
#     start_timeframe: Optional[str] = None  # When to start looking for slots
#     end_timeframe: Optional[str] = None    # When to stop looking for slots
#     duration_minutes: Optional[int] = None  # Duration of the meeting in minutes


# meeting_agent = Agent(
#     model,
#     output_type=MeetingTimeframe,
#     system_prompt=f"""
#     You are an expert at extracting meeting scheduling information from emails and messages.
    
#     Current datetime: {dt.strftime("%Y-%m-%dT%H:%M:%S%z")}
#     Current date: {datetime.now().strftime("%A")}
    
#     Your task is to analyze the content and extract meeting scheduling parameters:
    
#     1. start_timeframe: The earliest date to start looking for meeting slots
#        - For "today": use current date
#        - For "tomorrow": use next day
#        - For day names (Monday, Tuesday, etc.): find the NEXT occurrence of that day
#        - For "this week": use current date
#        - For "next week": use date of next Monday
#        - For specific dates: parse and use as-is
#        - Format: YYYY-MM-DDTHH:MM:SS+HH:MM
    
#     2. end_timeframe: The latest date to stop looking for meeting slots
#        - If specific day mentioned: use same day as start_timeframe
#        - If "this week" mentioned: use end of current week (Friday)
#        - If "next week" mentioned: use end of next week (Friday)  
#        - If range mentioned (e.g., "Monday to Wednesday"): use end of range
#        - Format: YYYY-MM-DDTHH:MM:SS+HH:MM
    
#     3. duration_minutes: Meeting duration in minutes
#        - Parse explicit durations: "30 minutes", "1 hour", "2 hours"
#        - Convert hours to minutes: 1 hour = 60 minutes
#        - Common defaults: "quick chat" = 15 min, "brief meeting" = 30 min
#        - If no duration specified: default to 60 minutes

#     ### ✅ Few-shot Examples (Context: Saturday, 19th July 2025)
    
#     #### 🗓️ Example 1
    
#     **Input:** `"Let's meet Thursday"`
        
#       "StartTime": "2025-07-24T00:00:00+05:30",
#       "EndTime": "2025-07-24T11:59:00+05:30",
#       "DurationMinutes": 60
        
#     #### 🗓️ Example 2
    
#     **Input:** `"30 minute call this week"`
        
#       "StartTime": "2025-07-19T00:00:00+05:30",
#       "EndTime": "2025-07-25T23:59:00+05:30",
#       "DurationMinutes": 30
        
#     #### 🗓️ Example 3
    
#     **Input:** `"2 hour session tomorrow"`
    
#       "StartTime": "2025-07-20T00:00:00+05:30",
#       "EndTime": "2025-07-20T23:59:00+05:30",
#       "DurationMinutes": 120
    
#     #### 🗓️ Example 4
    
#     **Input:** `"Meeting Monday to Wednesday"`
    
#       "StartTime": "2025-07-21T00:00:00+05:30",
#       "EndTime": "2025-07-23T23:59:00+05:30",
#       "DurationMinutes": 60

    
#     Be precise with date calculations and always consider the current date context.
#     """,
# )

In [142]:
# Define response model for meeting extraction
from pydantic import BaseModel
class MeetingTimeframe(BaseModel):
    start_timeframe: Optional[str] = None  # When to start looking for slots
    end_timeframe: Optional[str] = None    # When to stop looking for slots
    duration_minutes: Optional[int] = None  # Duration of the meeting in minutes
meeting_agent = Agent(
    model,
    output_type=MeetingTimeframe,
    system_prompt=f"""
    You are an expert at extracting meeting scheduling information from emails and messages.
    
    Current datetime: {datetime.now().strftime("%Y-%m-%dT%H:%M:%S%z")}
    Current date: {datetime.now().strftime("%A")}
    
    Your task is to analyze the content and extract meeting scheduling parameters:
    
    1. start_timeframe: The earliest date to start looking for meeting slots
       - For "today": use current date
       - For "tomorrow": use next day
       - For day names (Monday, Tuesday, etc.): find the NEXT occurrence of that day
       - For "this week": use current date
       - For "next week": use date of next Monday
       - For specific dates: parse and use as-is
       - Format: YYYY-MM-DDTHH:MM:SS+HH:MM
    
    2. end_timeframe: The latest date to stop looking for meeting slots
       - If specific day mentioned: use same day as start_timeframe
       - If "this week" mentioned: use end of current week (Friday)
       - If "next week" mentioned: use end of next week (Friday)  
       - If range mentioned (e.g., "Monday to Wednesday"): use end of range
       - Format: YYYY-MM-DDTHH:MM:SS+HH:MM
    
    3. duration_minutes: Meeting duration in minutes
       - Parse explicit durations: "30 minutes", "1 hour", "2 hours"
       - Convert hours to minutes: 1 hour = 60 minutes
       - Common defaults: "quick chat" = 15 min, "brief meeting" = 30 min
       - If no duration specified: default to 60 minutes
    ### ✅ Few-shot Examples (Context: Saturday, 19th July 2025)
    
    #### 🗓️ Example 1
    
    **Input:** `"Let's meet Thursday"`
        
      "StartTime": "2025-07-24T00:00:00+05:30",
      "EndTime": "2025-07-24T11:59:00+05:30",
      "DurationMinutes": 60
        
    #### 🗓️ Example 2
    
    **Input:** `"30 minute call this week"`
        
      "StartTime": "2025-07-19T00:00:00+05:30",
      "EndTime": "2025-07-25T23:59:00+05:30",
      "DurationMinutes": 30
        
    #### 🗓️ Example 3
    
    **Input:** `"2 hour session tomorrow"`
    
      "StartTime": "2025-07-20T00:00:00+05:30",
      "EndTime": "2025-07-20T23:59:00+05:30",
      "DurationMinutes": 120
    
    #### 🗓️ Example 4
    
    **Input:** `"Meeting Monday to Wednesday"`
    
      "StartTime": "2025-07-21T00:00:00+05:30",
      "EndTime": "2025-07-23T23:59:00+05:30",
      "DurationMinutes": 60
    
    Be precise with date calculations and always consider the current date context.
    """,
)


In [143]:
# Define response model for meeting extraction
from pydantic import BaseModel
class MeetingPrriority(BaseModel):
    priority: Optional[str] = None  # Priority Level Of Meeting
meeting_priority_agent = Agent(
    model,
    output_type=MeetingPrriority,
    system_prompt=f"""
    You are a calendar assistant that helps classify meeting priority. 
    Analyze calendar events and classify them as 'low', 'mid', or 'high' priority based on:

    PRIORITY GUIDELINES:
    
    HIGH PRIORITY:
    - Large meetings (5+ attendees) especially with external stakeholders
    - Executive/leadership meetings (words like: CEO, director, board, leadership, executive)
    - Client/customer meetings (words like: client, customer, demo, presentation, proposal)
    - Important project milestones (words like: launch, deadline, review, approval)
    - Interviews, performance reviews
    - Crisis/urgent meetings (words like: urgent, critical, emergency)
    
    MID PRIORITY:
    - Team meetings with 2-4 attendees
    - Project planning/status meetings
    - Regular recurring meetings (standup, weekly sync, planning)
    - Training sessions, workshops
    - 1:1 meetings with colleagues
    
    LOW PRIORITY:
    - Solo work time (SELF only) (When the only you are attending the event)
    - Optional meetings (words like: optional, FYI, social)
    - Personal time/breaks
    - Informal catch-ups
    - Large all-hands where your participation isn't critical

    Return only a JSON array with the priority for each event in the same order, like:
    ["high", "mid", "low"]
    """,
)


In [166]:
from datetime import datetime, timedelta
import ast
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

def merge_intervals(intervals):
    if not intervals:
        return []

    # Sort by start time
    intervals.sort(key=lambda x: x[0])
    merged = [intervals[0]]

    for current_start, current_end in intervals[1:]:
        last_start, last_end = merged[-1]
        if current_start <= last_end:
            merged[-1] = (last_start, max(last_end, current_end))  # Merge overlapping
        else:
            merged.append((current_start, current_end))
    return merged

# def retrieve_calendar_events(user, start, end):
#     events_list = []
#     token_path = "Keys/"+user.split("@")[0]+".token"
#     user_creds = Credentials.from_authorized_user_file(token_path)
#     calendar_service = build("calendar", "v3", credentials=user_creds)
#     events_result = calendar_service.events().list(
#         calendarId='primary', 
#         timeMin=start,
#         timeMax=end,
#         singleEvents=True,
#         orderBy='startTime'
#     ).execute()
#     events = events_result.get('items', [])
    
#     for event in events: 
#         attendee_list = []
#         try:
#             for attendee in event["attendees"]: 
#                 attendee_list.append(attendee['email'])
#         except: 
#             attendee_list.append("SELF")
        
#         start_time = event["start"]["dateTime"]
#         end_time = event["end"]["dateTime"]
#         events_list.append({
#             "StartTime": start_time, 
#             "EndTime": end_time, 
#             "NumAttendees": len(set(attendee_list)), 
#             "Attendees": list(set(attendee_list)),
#             "Summary": event["summary"]
#         })

#     print(events_list)
#     return events_list

import time
import asyncio
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials

import asyncio
import nest_asyncio
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials

# Enable nested event loops (useful in Jupyter notebooks)
nest_asyncio.apply()

async def get_priority_async(event_string):
    """Helper function to get priority asynchronously"""
    return await meeting_priority_agent.run(event_string)

def retrieve_calendar_events(user, start, end):
    events_list = []
    token_path = "Keys/"+user.split("@")[0]+".token"
    user_creds = Credentials.from_authorized_user_file(token_path)
    calendar_service = build("calendar", "v3", credentials=user_creds)
    events_result = calendar_service.events().list(
        calendarId='primary', 
        timeMin=start,
        timeMax=end,
        singleEvents=True,
        orderBy='startTime'
    ).execute()
    events = events_result.get('items', [])
    
    for event in events: 
        attendee_list = []
        try:
            for attendee in event["attendees"]: 
                attendee_list.append(attendee['email'])
        except: 
            attendee_list.append("SELF")
        
        start_time = event["start"]["dateTime"]
        end_time = event["end"]["dateTime"]
        
        # Create event data structure
        event_data = {
            "StartTime": start_time, 
            "EndTime": end_time, 
            "NumAttendees": len(set(attendee_list)), 
            "Attendees": list(set(attendee_list)),
            "Summary": event["summary"]
        }
        
        events_list.append(event_data)
    
    print(events_list)
    return events_list

def retrieve_calendar_events_priority(user, start, end, event_priority, num_attend):
    events_list = []
    token_path = "Keys/"+user.split("@")[0]+".token"
    user_creds = Credentials.from_authorized_user_file(token_path)
    calendar_service = build("calendar", "v3", credentials=user_creds)
    events_result = calendar_service.events().list(
        calendarId='primary', 
        timeMin=start,
        timeMax=end,
        singleEvents=True,
        orderBy='startTime'
    ).execute()
    events = events_result.get('items', [])
    
    # Define priority hierarchy for comparison
    priority_levels = {"low": 1, "mid": 2, "high": 3}
    min_priority_level = priority_levels.get(event_priority.lower(), 2)  # Default to "mid" if invalid
    
    for event in events: 
        attendee_list = []
        try:
            for attendee in event["attendees"]: 
                attendee_list.append(attendee['email'])
        except: 
            attendee_list.append("SELF")
        
        start_time = event["start"]["dateTime"]
        end_time = event["end"]["dateTime"]
        
        # Create event data structure
        event_data = {
            "StartTime": start_time, 
            "EndTime": end_time, 
            "NumAttendees": len(set(attendee_list)), 
            "Attendees": list(set(attendee_list)),
            "Summary": event["summary"]
        }
        
        # Get priority for this meeting
        try:
            # Convert event data to string format for the agent
            event_string = str(event_data)
            # Now asyncio.run() will work even in nested event loops
            priority_result = asyncio.run(get_priority_async(event_string))
            priority = priority_result.data.priority
        except Exception as e:
            print(f"Error getting priority for event '{event['summary']}': {e}")
            priority = "mid"  # Default priority if agent fails
        
        # Add priority to the event data
        event_data["Priority"] = priority
        
        # Only append if event priority is >= minimum threshold
        event_priority_level = priority_levels.get(priority.lower(), 2)  # Default to "mid"
        if event_priority_level > min_priority_level:
            events_list.append(event_data)
        elif event_priority_level == min_priority_level:
            if int(event_data["NumAttendees"]) >= num_attend:
                events_list.append(event_data)
    
    print(events_list)
    return events_list

# Usage remains the same:
# events = retrieve_calendar_events(user, start_date, end_date)

# Usage remains the same:
# events = retrieve_calendar_events(user, start_date, end_date)

# Usage example:
# async def main():
#     start_time = time.time()
#     events_with_priorities = await retrieve_calendar_events(
#         user="user@example.com", 
#         start="2025-07-22T00:00:00+05:30", 
#         end="2025-07-22T23:59:59+05:30",
#         meeting_priority_agent=meeting_priority_agent
#     )
#     end_time = time.time()
#     print(f"Processing time: {end_time - start_time} seconds")

def merge_intervals(intervals):
    if not intervals:
        return []

    # Sort by start time
    intervals.sort(key=lambda x: x[0])
    merged = [intervals[0]]

    for current_start, current_end in intervals[1:]:
        last_start, last_end = merged[-1]
        if current_start <= last_end:
            merged[-1] = (last_start, max(last_end, current_end))  # Merge overlapping
        else:
            merged.append((current_start, current_end))
    return merged

import ast

def parse_list(maybe_str):
    if isinstance(maybe_str, str):
        return ast.literal_eval(maybe_str)
    return maybe_str  # it's already a list

def get_free_time_slots(attendees, duration_minutes, start_date, end_date):
    user_availabilities = []
    users_schedule_dict = {}
    duration_minutes = int(duration_minutes)
    
    # Parse input dates
    start_dt = datetime.fromisoformat(start_date)
    end_dt = datetime.fromisoformat(end_date)
    
    # Input validation
    if start_dt >= end_dt:
        return "Error: Start date must be before end date", {}
    
    if duration_minutes <= 0:
        return "Error: Duration must be positive", {}
    
    attendees = parse_list(attendees)
    if not attendees:
        return "Error: No attendees provided", {}
    
    # Collect all busy periods and build users_schedule_dict
    for user in attendees:
        users_schedule_dict[user] = []
        try:
            events = retrieve_calendar_events(user, start_date, end_date)
            for event in events:
                event_start = datetime.fromisoformat(event["StartTime"])
                event_end = datetime.fromisoformat(event["EndTime"])
                
                # Store original event info in users_schedule_dict
                users_schedule_dict[user].append(event)
                
                # Clip events to the search window for availability calculation
                clipped_start = max(event_start, start_dt)
                clipped_end = min(event_end, end_dt)
                
                # Only add if there's actual overlap with our time window
                if clipped_start < clipped_end:
                    user_availabilities.append((clipped_start, clipped_end))
        except Exception as e:
            print(f"Warning: Could not retrieve events for {user}: {e}")
            continue
    
    # Handle case where no events found
    if not user_availabilities:
        return f"slot 1 : {start_date} - {end_date}\n ", users_schedule_dict
    
    merged_unavailable = merge_intervals(user_availabilities)
    available = []
    duration = timedelta(minutes=duration_minutes)
    
    # Check slot before first busy period
    if len(merged_unavailable) > 0:
        gap_before = merged_unavailable[0][0] - start_dt
        if gap_before >= duration:
            available.append((start_dt, merged_unavailable[0][0]))
    
    # Check gaps between busy periods
    for i in range(1, len(merged_unavailable)):
        gap_duration = merged_unavailable[i][0] - merged_unavailable[i-1][1]
        if gap_duration >= duration:
            available.append((merged_unavailable[i-1][1], merged_unavailable[i][0]))
    
    # Check slot after last busy period
    if len(merged_unavailable) > 0:
        gap_after = end_dt - merged_unavailable[-1][1]
        if gap_after >= duration:
            available.append((merged_unavailable[-1][1], end_dt))
    
    # Format output consistently (using isoformat to match your end_date parameter)
    if not available:
        return "No available time slots found for the specified duration\n ", users_schedule_dict
    
    slots = ""
    idx = 1
    for slot_start, slot_end in available:
        # Use consistent isoformat for both start and end
        start_str = slot_start.isoformat()
        end_str = slot_end.isoformat()
        slots += f"slot {idx} : {start_str} - {end_str}\n "
        idx += 1
    
    return slots, users_schedule_dict

def get_free_time_slots_priority(attendees, duration_minutes, start_date, end_date, event_priority, num_attend):
    user_availabilities = []
    duration_minutes = int(duration_minutes)
    
    # Parse input dates
    start_dt = datetime.fromisoformat(start_date)
    end_dt = datetime.fromisoformat(end_date)
    
    # Input validation
    if start_dt >= end_dt:
        return "Error: Start date must be before end date"
    
    if duration_minutes <= 0:
        return "Error: Duration must be positive"
    
    attendees = parse_list(attendees)
    if not attendees:
        return "Error: No attendees provided"
    
    # Collect all busy periods
    for user in attendees:
        try:
            events = retrieve_calendar_events_priority(user, start_date, end_date, event_priority, num_attend)
            for event in events:
                event_start = datetime.fromisoformat(event["StartTime"])
                event_end = datetime.fromisoformat(event["EndTime"])
                
                # Clip events to the search window
                clipped_start = max(event_start, start_dt)
                clipped_end = min(event_end, end_dt)
                
                # Only add if there's actual overlap with our time window
                if clipped_start < clipped_end:
                    user_availabilities.append((clipped_start, clipped_end))
        except Exception as e:
            print(f"Warning: Could not retrieve events for {user}: {e}")
            continue
    
    # Handle case where no events found
    if not user_availabilities:
        return f"slot 1 : {start_date} - {end_date}\n "
    
    merged_unavailable = merge_intervals(user_availabilities)
    available = []
    duration = timedelta(minutes=duration_minutes)
    
    # Check slot before first busy period
    if len(merged_unavailable) > 0:
        gap_before = merged_unavailable[0][0] - start_dt
        if gap_before >= duration:
            available.append((start_dt, merged_unavailable[0][0]))
    
    # Check gaps between busy periods
    for i in range(1, len(merged_unavailable)):
        gap_duration = merged_unavailable[i][0] - merged_unavailable[i-1][1]
        if gap_duration >= duration:
            available.append((merged_unavailable[i-1][1], merged_unavailable[i][0]))
    
    # Check slot after last busy period
    if len(merged_unavailable) > 0:
        gap_after = end_dt - merged_unavailable[-1][1]
        if gap_after >= duration:
            available.append((merged_unavailable[-1][1], end_dt))
    
    # Format output consistently (using isoformat to match your end_date parameter)
    if not available:
        return "No available time slots found for the specified duration\n "
    
    slots = ""
    idx = 1
    for slot_start, slot_end in available:
        # Use consistent isoformat for both start and end
        start_str = slot_start.isoformat()
        end_str = slot_end.isoformat()
        slots += f"slot {idx} : {start_str} - {end_str}\n "
        idx += 1
    
    return slots

In [145]:
data = {
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033d5",
    "Datetime": "19-07-2025T12:34:55",
    "Location": "IISc Bangalore",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "Subject": "Client Feedback",
    "EmailContent": "Hi Team. We’ve received the final feedback from the client. Let’s review it together and plan next steps. Let's meet on Wednesday at 10:00 A.M."
}

In [146]:
priority_result = asyncio.run(get_priority_async(data["EmailContent"]))
priority = priority_result.data.priority

/tmp/ipykernel_16626/3164075084.py:2: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  priority = priority_result.data.priority


In [147]:
data["Priority"] = priority
priority

'high'

In [148]:
import  time

start_time = time.time()
result = await meeting_agent.run(data["EmailContent"])

# Convert to dictionary for JSON serialization
response_data = {
    "start_timeframe": result.data.start_timeframe,
    "end_timeframe": result.data.end_timeframe,
    "duration_minutes": result.data.duration_minutes,
    "success": True,
    "message": "Successfully extracted meeting timeframes"
}

/tmp/ipykernel_16626/1036837732.py:8: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "start_timeframe": result.data.start_timeframe,
/tmp/ipykernel_16626/1036837732.py:9: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "end_timeframe": result.data.end_timeframe,
/tmp/ipykernel_16626/1036837732.py:10: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "duration_minutes": result.data.duration_minutes,


In [149]:
if data["Priority"] == 'high':
    from datetime import datetime, timedelta
    # Parse the start time
    start_time = datetime.fromisoformat(response_data['start_timeframe'])
    
    # Add duration to get end time
    end_time = start_time + timedelta(minutes=response_data['duration_minutes'])
    
    # Update the response_data
    response_data['end_timeframe'] = end_time.isoformat()
    

In [150]:
response_data

{'start_timeframe': '2025-07-23T10:00:00+05:30',
 'end_timeframe': '2025-07-23T11:00:00+05:30',
 'duration_minutes': 60,
 'success': True,
 'message': 'Successfully extracted meeting timeframes'}

In [151]:
def get_attendees_with_from(data):
    """
    Extract attendees emails as a list along with From email.
    If Attendees is just ['SELF'], return only the From email.
    
    Args:
        data (dict): The input data containing From and Attendees
        
    Returns:
        list: List of email addresses
    """
    from_email = data.get("From", "")
    attendees = data.get("Attendees", [])
    
    # Extract emails from attendees list
    attendee_emails = []
    for attendee in attendees:
        if isinstance(attendee, dict) and "email" in attendee:
            attendee_emails.append(attendee["email"])
        elif isinstance(attendee, str):
            attendee_emails.append(attendee)
    
    # Special case: if attendees is just ['SELF'], return only From email
    if attendee_emails == ['SELF']:
        return [from_email]
    
    # Otherwise, combine From email with attendees emails
    all_emails = [from_email] + attendee_emails
    
    # Remove duplicates while preserving order
    seen = set()
    result = []
    for email in all_emails:
        if email and email not in seen:
            seen.add(email)
            result.append(email)
    
    return result


# Test normal case
attendees = get_attendees_with_from(data)
print(attendees)

['userone.amd@gmail.com', 'usertwo.amd@gmail.com', 'userthree.amd@gmail.com']


In [152]:
free_slots, user_schedule_dict = get_free_time_slots(attendees, response_data["duration_minutes"], response_data["start_timeframe"], response_data["end_timeframe"])

[]
[]
[{'StartTime': '2025-07-23T10:00:00+05:30', 'EndTime': '2025-07-23T11:00:00+05:30', 'NumAttendees': 1, 'Attendees': ['SELF'], 'Summary': 'Client Meeting - IMPORTANT'}]
[{'StartTime': '2025-07-23T10:00:00+05:30', 'EndTime': '2025-07-23T11:00:00+05:30', 'NumAttendees': 1, 'Attendees': ['SELF'], 'Summary': 'Client Meeting - IMPORTANT'}]
[]
[]


In [153]:
free_slots

'No available time slots found for the specified duration\n '

In [154]:
if free_slots == 'No available time slots found for the specified duration\n ':
    free_slots = get_free_time_slots_priority(attendees, response_data["duration_minutes"], response_data["start_timeframe"], response_data["end_timeframe"], data["Priority"], len(data["Attendees"]))

[]


/tmp/ipykernel_16626/2438647515.py:157: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  priority = priority_result.data.priority


[]
[]


In [155]:
free_slots

'slot 1 : 2025-07-23T10:00:00+05:30 - 2025-07-23T11:00:00+05:30\n '

In [156]:
# Define response model for meeting extraction
from pydantic import BaseModel
from typing import Optional

class OptimalTime(BaseModel):
    start_time_of_meeting: Optional[str] = None  # Meeting Start
    end_time_of_meeting: Optional[str] = None    # Meeting End
    duration_minutes: Optional[int] = None  # Duration of the meeting in minutes
    metadata: Optional[str] = None # Why I Chose The Time

slots_agents = Agent(
    model,
    output_type=OptimalTime,
    system_prompt=f"""
    Available Free Slots Common Between All The Attendees Will Be Given
    Duration Of The Meeting: Will Be Given

    Return Should Have
    start_timeframe:Meeting Start Time
    end_timeframe:Meeting End Time
    duration_minutes:Duration of the meeting in minutes
    metadata:Why You Chose The Time

    Note: Meeting Start - End Time should of given duration_minutes
    
    Return the optimal meeting time that maximizes convenience for all attendees while respecting the duration requirements.
    """,
)

In [157]:
import  time

start_time = time.time()
result = await slots_agents.run(f"Free Slots {free_slots} Duration: {response_data["duration_minutes"]}")

# Convert to dictionary for JSON serialization
response_data = {
    "start_time_of_meeting": result.data.start_time_of_meeting,
    "end_time_of_meeting": result.data.end_time_of_meeting,
    "duration_minutes": result.data.duration_minutes,
    "metadata": result.data.metadata
}

/tmp/ipykernel_16626/3296042613.py:8: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "start_time_of_meeting": result.data.start_time_of_meeting,
/tmp/ipykernel_16626/3296042613.py:9: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "end_time_of_meeting": result.data.end_time_of_meeting,
/tmp/ipykernel_16626/3296042613.py:10: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "duration_minutes": result.data.duration_minutes,
/tmp/ipykernel_16626/3296042613.py:11: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "metadata": result.data.metadata


In [158]:
response_data

{'start_time_of_meeting': '2025-07-23T10:00:00+05:30',
 'end_time_of_meeting': '2025-07-23T11:00:00+05:30',
 'duration_minutes': 60,
 'metadata': 'The meeting is scheduled for the entire duration of the provided free slot.'}

In [185]:
import time
async def  your_meeting_assistant(data): 

    # Start timer
    start = time.time()
    
    # get priority of meeting
    priority_result = asyncio.run(get_priority_async(data["EmailContent"]))
    priority = priority_result.data.priority
    data["Priority"] = priority

    # End timer
    end = time.time()
    
    # Calculate and print runtime
    runtime = end - start
    print(f"get_priority_async Runtime: {runtime:.4f} seconds")

    # Start timer
    start = time.time()
    
    #get the timeframe Entire available timeframe not just meeting timeframe
    meeting_agent_results = await meeting_agent.run(data["EmailContent"])
    response_data_meeting = {
        "start_timeframe": meeting_agent_results.data.start_timeframe,
        "end_timeframe": meeting_agent_results.data.end_timeframe,
        "duration_minutes": meeting_agent_results.data.duration_minutes,
        "success": True,
        "message": "Successfully extracted meeting timeframes"
    }
    
    # End timer
    end = time.time()

    # Calculate and print runtime
    runtime = end - start
    print(f"meeting_agent Runtime: {runtime:.4f} seconds")

    print("EXTRACTED TIME ELEMENTS")
    print(response_data_meeting)


    if data["Priority"] == 'high':
        start_time = datetime.fromisoformat(response_data_meeting['start_timeframe'])
        end_time = start_time + timedelta(minutes=response_data_meeting['duration_minutes'])
        response_data_meeting['end_timeframe'] = end_time.isoformat()
        

    #get Attendees
    print("Extracted Attendees")
    attendees = get_attendees_with_from(data)
    print(attendees)

    # Start timer
    start = time.time()

    free_slots, user_schedule_dict = get_free_time_slots(attendees, response_data_meeting["duration_minutes"], response_data_meeting["start_timeframe"], response_data_meeting["end_timeframe"])
    print("Extracted Free Slots")
    if free_slots == 'No available time slots found for the specified duration\n ':
        print('No available time slots found for the specified duration\n ')
        free_slots = get_free_time_slots_priority(attendees, response_data_meeting["duration_minutes"], response_data_meeting["start_timeframe"], response_data_meeting["end_timeframe"], data["Priority"], len(data["Attendees"]))
    print(free_slots)

    # End timer
    end = time.time()
    
    # Calculate and print runtime
    runtime = end - start
    print(f"get_free_time_slots Runtime: {runtime:.4f} seconds")

    # Start timer
    start = time.time()

    slots_agent_results =  await slots_agents.run(f"Free Slots {free_slots} Duration: {response_data_meeting["duration_minutes"]}")
    response_data_slots = {
        "start_time_of_meeting": slots_agent_results.data.start_time_of_meeting,
        "end_time_of_meeting": slots_agent_results.data.end_time_of_meeting,
        "duration_minutes": slots_agent_results.data.duration_minutes,
        "metadata": slots_agent_results.data.metadata
    }
    print("Suggested Slot")
    print(response_data_slots)

    # End timer
    end = time.time()
    
    # Calculate and print runtime
    runtime = end - start
    print(f"slots_agent Runtime: {runtime:.4f} seconds")

    #parsing the response
    raw = response_data_slots["start_time_of_meeting"].strip()
    try:
        dt = parser.isoparse(raw)  # Faster, stricter ISO parser
    except Exception:
        dt = parser.parse(raw) 
    available_start_time = dt.strftime("%Y-%m-%dT%H:%M:%S%z")
    available_end_dt = dt + timedelta(minutes=response_data_meeting["duration_minutes"])
    available_end_time = available_end_dt.strftime("%Y-%m-%dT%H:%M:%S%z")

    
    output = {}
    output["Request_id"] = data["Request_id"]
    output["Datetime"] = data["Datetime"]
    output["Location"] = data["Location"]
    output["Request_id"] = data["Request_id"]
    output["From"] = data["From"]
    output["Location"] = data["Location"]
    output["Attendees"] = data["Attendees"]
    output["Subject"] = data["Subject"]
    output["EmailContent"] = data["EmailContent"]
    output["EventStart"] = available_start_time
    output["EventEnd"] = available_end_time
    output["Duration_mins"] =  str(response_data_meeting["duration_minutes"])
    output["MetaData"] = response_data_slots['metadata']
    for i  in range(len(output["Attendees"])):
        email = output["Attendees"][i]["email"]
        output["Attendees"][i]["events"] = user_schedule_dict[email]
    return output

In [179]:
data = {
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
    "Datetime": "19-07-2025T12:34:55",
    "Location": "IISc Bangalore",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "Subject": "Agentic AI Project Status Update",
    "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."
}


In [180]:

import time

# Start timer
start = time.time()

# Run your function here
output = await your_meeting_assistant(data)

pprint(output)

# End timer
end = time.time()

# Calculate and print runtime
runtime = end - start
print(f"Runtime: {runtime:.4f} seconds")

/tmp/ipykernel_16626/46396761.py:9: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  priority = priority_result.data.priority


get_priority_async Runtime: 0.9495 seconds


/tmp/ipykernel_16626/46396761.py:25: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "start_timeframe": meeting_agent_results.data.start_timeframe,
/tmp/ipykernel_16626/46396761.py:26: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "end_timeframe": meeting_agent_results.data.end_timeframe,
/tmp/ipykernel_16626/46396761.py:27: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "duration_minutes": meeting_agent_results.data.duration_minutes,


meeting_agent Runtime: 2.5299 seconds
EXTRACTED TIME ELEMENTS
{'start_timeframe': '2025-07-24T00:00:00+05:30', 'end_timeframe': '2025-07-24T23:59:00+05:30', 'duration_minutes': 30, 'success': True, 'message': 'Successfully extracted meeting timeframes'}
Extracted Attendees
['userone.amd@gmail.com', 'usertwo.amd@gmail.com', 'userthree.amd@gmail.com']
[{'StartTime': '2025-07-23T18:00:00+05:30', 'EndTime': '2025-07-24T09:00:00+05:30', 'NumAttendees': 1, 'Attendees': ['SELF'], 'Summary': 'Off Hours'}, {'StartTime': '2025-07-24T18:00:00+05:30', 'EndTime': '2025-07-25T09:00:00+05:30', 'NumAttendees': 1, 'Attendees': ['SELF'], 'Summary': 'Off Hours'}]
[{'StartTime': '2025-07-23T18:00:00+05:30', 'EndTime': '2025-07-24T09:00:00+05:30', 'NumAttendees': 1, 'Attendees': ['SELF'], 'Summary': 'Off Hours'}, {'StartTime': '2025-07-24T18:00:00+05:30', 'EndTime': '2025-07-25T09:00:00+05:30', 'NumAttendees': 1, 'Attendees': ['SELF'], 'Summary': 'Off Hours'}]
[{'StartTime': '2025-07-23T16:00:00+05:30', 'E

/tmp/ipykernel_16626/46396761.py:76: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "start_time_of_meeting": slots_agent_results.data.start_time_of_meeting,
/tmp/ipykernel_16626/46396761.py:77: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "end_time_of_meeting": slots_agent_results.data.end_time_of_meeting,
/tmp/ipykernel_16626/46396761.py:78: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "duration_minutes": slots_agent_results.data.duration_minutes,
/tmp/ipykernel_16626/46396761.py:79: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "metadata": slots_agent_results.data.metadata


In [188]:
import time
import asyncio
from datetime import datetime, timedelta
from dateutil import parser

# Alternative version with even more optimization
async def your_meeting_assistant_ultra_optimized(data): 
    # Start timer for all concurrent operations
    start_total = time.time()

    # Start timer
    start = time.time()
    
    # Create tasks for all independent operations
    priority_task = get_priority_async(data["EmailContent"])
    meeting_agent_task = meeting_agent.run(data["EmailContent"])

    # End timer
    end = time.time()
    
    # Calculate and print runtime
    runtime = end - start
    print(f"meeting_agent Runtime: {runtime:.4f} seconds")
    

    # Start timer
    start = time.time()
    
    # Get attendees synchronously (assuming this is fast)
    attendees = get_attendees_with_from(data)
    print("Extracted Attendees")
    print(attendees)

    # End timer
    end = time.time()
    
    # Calculate and print runtime
    runtime = end - start
    print(f"get_attendees_with_from Runtime: {runtime:.4f} seconds")
    
    # Wait for priority and meeting agent results
    priority_result, meeting_agent_results = await asyncio.gather(
        priority_task,
        meeting_agent_task
    )
    
    # Process results
    priority = priority_result.data.priority
    data["Priority"] = priority
    
    response_data_meeting = {
        "start_timeframe": meeting_agent_results.data.start_timeframe,
        "end_timeframe": meeting_agent_results.data.end_timeframe,
        "duration_minutes": meeting_agent_results.data.duration_minutes,
        "success": True,
        "message": "Successfully extracted meeting timeframes"
    }
    
    print("EXTRACTED TIME ELEMENTS")
    print(response_data_meeting)

    # Handle high priority meetings
    if priority == 'high':
        start_time = datetime.fromisoformat(response_data_meeting['start_timeframe'])
        end_time = start_time + timedelta(minutes=response_data_meeting['duration_minutes'])
        response_data_meeting['end_timeframe'] = end_time.isoformat()
    
    # Get free slots
    free_slots, user_schedule_dict = get_free_time_slots(
        attendees, 
        response_data_meeting["duration_minutes"], 
        response_data_meeting["start_timeframe"], 
        response_data_meeting["end_timeframe"]
    )
    
    print("Extracted Free Slots")
    if free_slots == 'No available time slots found for the specified duration\n ':
        print('No available time slots found for the specified duration\n ')
        free_slots = get_free_time_slots_priority(
            attendees, 
            response_data_meeting["duration_minutes"], 
            response_data_meeting["start_timeframe"], 
            response_data_meeting["end_timeframe"], 
            priority, 
            len(data["Attendees"])
        )
    print(free_slots)

    # Get slots suggestion
    slots_agent_results = await slots_agents.run(
        f"Free Slots {free_slots} Duration: {response_data_meeting['duration_minutes']}"
    )
    
    response_data_slots = {
        "start_time_of_meeting": slots_agent_results.data.start_time_of_meeting,
        "end_time_of_meeting": slots_agent_results.data.end_time_of_meeting,
        "duration_minutes": slots_agent_results.data.duration_minutes,
        "metadata": slots_agent_results.data.metadata
    }
    
    print("Suggested Slot")
    print(response_data_slots)

    # Parse the response
    raw = response_data_slots["start_time_of_meeting"].strip()
    try:
        dt = parser.isoparse(raw)
    except Exception:
        dt = parser.parse(raw) 
    
    available_start_time = dt.strftime("%Y-%m-%dT%H:%M:%S%z")
    available_end_dt = dt + timedelta(minutes=response_data_meeting["duration_minutes"])
    available_end_time = available_end_dt.strftime("%Y-%m-%dT%H:%M:%S%z")

    # End total timer
    end_total = time.time()
    total_runtime = end_total - start_total
    print(f"Total optimized runtime: {total_runtime:.4f} seconds")

    # Build output efficiently
    output = {
        "Request_id": data["Request_id"],
        "Datetime": data["Datetime"],
        "Location": data["Location"],
        "From": data["From"],
        "Attendees": [
            {**attendee, "events": user_schedule_dict[attendee["email"]]} 
            for attendee in data["Attendees"]
        ],
        "Subject": data["Subject"],
        "EmailContent": data["EmailContent"],
        "EventStart": available_start_time,
        "EventEnd": available_end_time,
        "Duration_mins": str(response_data_meeting["duration_minutes"]),
        "MetaData": response_data_slots['metadata']
    }
    
    return output

In [189]:

import time

# Start timer
start = time.time()

# Run your function here
output = await your_meeting_assistant_ultra_optimized(data)

pprint(output)

# End timer
end = time.time()

# Calculate and print runtime
runtime = end - start
print(f"Runtime: {runtime:.4f} seconds")

meeting_agent Runtime: 0.0000 seconds
Extracted Attendees
['userone.amd@gmail.com', 'usertwo.amd@gmail.com', 'userthree.amd@gmail.com']
get_attendees_with_from Runtime: 0.0000 seconds


/tmp/ipykernel_16626/1423164296.py:48: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  priority = priority_result.data.priority
/tmp/ipykernel_16626/1423164296.py:52: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "start_timeframe": meeting_agent_results.data.start_timeframe,
/tmp/ipykernel_16626/1423164296.py:53: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "end_timeframe": meeting_agent_results.data.end_timeframe,
/tmp/ipykernel_16626/1423164296.py:54: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "duration_minutes": meeting_agent_results.data.duration_minutes,


EXTRACTED TIME ELEMENTS
{'start_timeframe': '2025-07-24T00:00:00+05:30', 'end_timeframe': '2025-07-24T23:59:00+05:30', 'duration_minutes': 30, 'success': True, 'message': 'Successfully extracted meeting timeframes'}
[{'StartTime': '2025-07-23T18:00:00+05:30', 'EndTime': '2025-07-24T09:00:00+05:30', 'NumAttendees': 1, 'Attendees': ['SELF'], 'Summary': 'Off Hours'}, {'StartTime': '2025-07-24T18:00:00+05:30', 'EndTime': '2025-07-25T09:00:00+05:30', 'NumAttendees': 1, 'Attendees': ['SELF'], 'Summary': 'Off Hours'}]
[{'StartTime': '2025-07-23T18:00:00+05:30', 'EndTime': '2025-07-24T09:00:00+05:30', 'NumAttendees': 1, 'Attendees': ['SELF'], 'Summary': 'Off Hours'}, {'StartTime': '2025-07-24T18:00:00+05:30', 'EndTime': '2025-07-25T09:00:00+05:30', 'NumAttendees': 1, 'Attendees': ['SELF'], 'Summary': 'Off Hours'}]
[{'StartTime': '2025-07-23T16:00:00+05:30', 'EndTime': '2025-07-24T07:30:00+05:30', 'NumAttendees': 1, 'Attendees': ['SELF'], 'Summary': 'Off Hours'}, {'StartTime': '2025-07-24T16:00

/tmp/ipykernel_16626/1423164296.py:95: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "start_time_of_meeting": slots_agent_results.data.start_time_of_meeting,
/tmp/ipykernel_16626/1423164296.py:96: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "end_time_of_meeting": slots_agent_results.data.end_time_of_meeting,
/tmp/ipykernel_16626/1423164296.py:97: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "duration_minutes": slots_agent_results.data.duration_minutes,
/tmp/ipykernel_16626/1423164296.py:98: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "metadata": slots_agent_results.data.metadata
